#   Workflow use example

This section provides an example of how to build a workflow using the API. In this example, FloatInput based nodes can be plugged in output nodes, the output nodes will display the sum of the plugged float input nodes.

First the necessary classes must be imported from the module:


In [ ]:
from typing import Any, Dict, List

import panel as pn
pn.extension()

from panel_reactflow.workflow import Workflow, WorkflowNode
from panel_reactflow.api import NodePort, PortDirection, PortPosition, Node, Edge

##  Nodes definition


The float input node is defined by the code below. 

-   A ``pn.widgets.FloatInput`` object is created and stored as class attribute. If its value change, the ``update`` function is called. 
-   In this case, the ``update`` function only needs to trigger its output nodes (nodes plugged to output ports).
-   The node has only one port, which is an output, which we decide to locate on the node right.
-   The ``create`` function returns a column that centers the ``FloatInput`` widget.
-   The node dictionary returned by ``get_node_json_value`` contains the ``FloatInput`` value that we set at the "value" key.


In [ ]:
class FloatInputNode(WorkflowNode):
    node_class_name = "Float Input"
    ports:List[NodePort] = [NodePort(direction=PortDirection.OUTPUT, position=PortPosition.RIGHT, name="output")]

    def __init__(self, ):
        super().__init__()
        self.float_input = pn.widgets.FloatInput(value=0., width=100)
        
        self.float_input.param.watch(self.update, "value")

    def create(self, ):
        return pn.layout.Column(
                                    self.float_input, 
                                    name=self.name, 
                                    align="center"
                                )
    
    def update(self, _):
        self.update_outputs()

    def get_node_json_value(self) -> Dict[str, Any]:
        return {"value" : self.float_input.value}


The result node input node is defined by the code below. 

-   The inputs sum is displayed in a  ``pn.pane.Markdown`` object that is created and stored as class attribute. 
-   The node has one input port, which we decide to locate on the node left to be consistent with the float input node.
-   The ``create`` function returns a column that centers the ``Markdown`` widget.
-   In this case, the ``update`` function reads the list of nodes plugged at its ``input`` port:
    -   if no node is plugged, a "Result : Undefined" message is displayed
    -   if nodes are plugged, their value is summed and displayed in the ``Markdown`` widget.


In [ ]:
class ResultNode(WorkflowNode):
    node_class_name = "Result"
    ports:List[NodePort] = [NodePort(direction=PortDirection.INPUT, position=PortPosition.LEFT, name="input")]

    def __init__(self, ):
        super().__init__()
        self.result_label = pn.pane.Markdown("Result : Undefined")

    def create(self, ):
        return pn.layout.Column(
                                    self.result_label, 
                                    name=self.name, 
                                    align="center"
                                )
    
    def update(self, _):
        value = 0

        if "input" in self.plugged_nodes :
            if len(self.plugged_nodes["input"]) == 0:
                self.result_label.object = f"Result : Undefined"
            else:
                for float_input in self.plugged_nodes["input"]:
                    value += float_input.get_node_json_value()["value"]

                self.result_label.object = f"Addition result : {round(value, 1)}"
            self.update_outputs()

    def get_node_json_value(self) -> Dict[str, Any]:
        return {"value" : self.result_label.object}


##  Create the workflow graph

Now the both node classes were created, a Workflow object can be created providing the node classes in arguments.

In [ ]:
empty_graph = Workflow(nodes_classes = [FloatInputNode, ResultNode],
                    initial_nodes=[],
                    initial_edges=[])

In [ ]:
pn.Row(empty_graph, height=600)

A reactlow graph can also be created with initial nodes and edges defined.

In [ ]:
node_1 = FloatInputNode()
node_2 = ResultNode()

filled_graph = Workflow(nodes_classes = [FloatInputNode, ResultNode],
                    initial_nodes=[
                        Node("Node_1", node_1, 0, 0),
                        Node("Node_2", node_2, 200, 0),
                    ],
                    initial_edges=[
                        Edge("Node_1", "output", "Node_2", "input"),
                    ])

In [ ]:
pn.Row(filled_graph, height=600)